In [12]:
import pickle
from collections import Counter
from tqdm import tqdm
import pandas as pd

In [2]:
DATASET='/home/kvassay/data/z/data/reviews_train_test_dev1_{}.pickle'
TYPE='tok'

In [3]:
%%time
with open(DATASET.format(TYPE),'rb') as f:
    train,_,_=pickle.load(f)

CPU times: user 5.43 s, sys: 1.12 s, total: 6.55 s
Wall time: 6.54 s


In [6]:
lengths=[len(x['text']) for x in train]

In [13]:
pd.DataFrame(lengths).describe()

,0
count,551399.000000
mean,95.317418
std,94.979352
min,3.000000
25%,40.000000
50%,67.000000
75%,116.000000
max,4130.000000
